<a href="https://colab.research.google.com/github/RayNCode/code_collab/blob/main/Main_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I try to compile everything I have done so far. I merged all the datasets together and I'm trying to keep only the relevant variables for our model.

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [6]:
learn_dataset_job = pd.read_csv("/content/learn_dataset_job.csv")
work_desc = pd.read_csv("/content/code_work_description_map.csv")
merged_df = pd.merge(learn_dataset_job, work_desc, left_on='work_description', right_on='N3', how='inner')
print(merged_df.shape)
learn_dataset = pd.read_csv("/content/learn_dataset.csv")
data_2 = pd.merge(learn_dataset, merged_df, on="Id", how="left")
print(data_2.shape)
dept_code = pd.read_csv("/content/city_adm.csv")
df_1 = data_2.merge(dept_code, on='insee_code')
print(df_1.shape)
emp_contract = pd.read_csv("/content/learn_dataset_Emp_contract.csv")
df_2 = df_1.merge(emp_contract, on='Id', how='left')
print(df_2.shape)
learn_dataset_sport = pd.read_csv("/content/learn_dataset_sport.csv")
df_3 = df_2.merge(learn_dataset_sport, on='Id', how='left')
print(df_3.shape)
code_Club = pd.read_csv("/content/code_Club.csv")
df = df_3.merge(code_Club, left_on='Club', right_on='Code', how='left')
df.shape

(19279, 14)
(49993, 23)
(49993, 26)
(49993, 27)
(49993, 28)


(49993, 32)

In [17]:
df['Categorie'] = df['Categorie'].astype('object')
print(df.dtypes)


Id                     int64
insee_code            object
is_student              bool
OCCUPATION_42         object
ACTIVITY_TYPE         object
AGE_2019               int64
household             object
Highest_degree        object
sex                   object
target                object
employer_category     object
job_category          object
EMPLOYEE_COUNT        object
Terms_of_emp          object
Eco_sect              object
work_description      object
Job_dep               object
Working_hours        float64
WORK_CONDITION        object
EMOLUMENT            float64
N3                    object
N2                   float64
N1                   float64
Nom de la commune     object
town_type             object
dep                   object
Emp_contract          object
Club                  object
Code                  object
Categorie             object
Nom fédération        object
Nom catégorie         object
dtype: object


In [ ]:
# TYPE2|2	Elèves, étudiants, stagiaires non rémunéré de 14 ans ou plus
# TYPE2|3	Moins de 14 ans

# Je dois trouver un moyen de parametrer les emoluements et autres variables dépendantes du travail à 0 et ajouter une autre catégorie "None".
# Peut-être le faire après la prédiction justement comme ça on garde le reste.

In [7]:
categorical_columns = df.select_dtypes(include=['object']).columns
# Remplacer les valeurs manquantes par "None" dans les colonnes catégorielles
df[categorical_columns] = df[categorical_columns].fillna("None")


In [10]:
y = df['target'].copy()
y = np.where(y == 'B', 1, 0)
X = df.drop(["Id", "target", 'insee_code'], axis="columns").copy()

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Sélectionnez les colonnes numériques pour l'imputation
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Créez l'IterativeImputer
imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)

# Appliquez l'imputation
X[numeric_columns] = imputer.fit_transform(X[numeric_columns])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [13]:
X.columns

Index(['is_student', 'OCCUPATION_42', 'ACTIVITY_TYPE', 'AGE_2019', 'household',
       'Highest_degree', 'sex', 'employer_category', 'job_category',
       'EMPLOYEE_COUNT', 'Terms_of_emp', 'Eco_sect', 'work_description',
       'Job_dep', 'Working_hours', 'WORK_CONDITION', 'EMOLUMENT', 'N3', 'N2',
       'N1', 'Nom de la commune', 'town_type', 'dep', 'Emp_contract', 'Club',
       'Code', 'Categorie', 'Nom fédération', 'Nom catégorie'],
      dtype='object')

In [14]:
X

,is_student,OCCUPATION_42,ACTIVITY_TYPE,AGE_2019,household,Highest_degree,sex,employer_category,job_category,EMPLOYEE_COUNT,...,N1,Nom de la commune,town_type,dep,Emp_contract,Club,Code,Categorie,Nom fédération,Nom catégorie
0,True,csp_6_8,TYPE1|1,20.0,TYPMR1-1,edu.1.4,Male,ct_9,A,tr_3,...,6.0,L'Abergement-Clémenciat,Commune simple,01,COND.1.1,None,None,1.01,None,None
1,False,csp_6_5,TYPE1|1,48.0,TYPMR4-1,edu.1.3,Male,ct_9,O,tr_5,...,6.0,Ambérieu-en-Bugey,Chef-lieu canton,01,COND.1.6,fede.109,fede.109,1.00,Fédération Française d'équitation,Fédération unisport olympique
2,False,csp_8_1,TYPE1|2,37.0,TYPMR4-4,edu.1.8,Female,None,None,None,...,5.0,Ambérieu-en-Bugey,Chef-lieu canton,01,None,fede.102,fede.102,1.00,Fédération Française des sociétés d'aviron,Fédération unisport olympique
3,False,csp_4_7,TYPE1|2,65.0,TYPMR3-1,edu.1.6,Male,None,None,None,...,5.0,Ambérieu-en-Bugey,Chef-lieu canton,01,None,None,None,1.00,None,None
4,True,csp_8_4,TYPE2|2,22.0,TYPMR1-2,edu.1.4,Female,None,None,None,...,5.0,Ambérieu-en-Bugey,Chef-lieu canton,01,None,None,None,2.00,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49988,False,csp_4_6,TYPE1|1,57.0,TYPMR1-2,edu.1.3,Female,ct_4,O,tr_5,...,4.0,Villiers-le-Bel,Chef-lieu canton,95,COND.1.6,fede.103,fede.103,1.00,Fédération Française de badminton,Fédération unisport olympique
49989,False,csp_8_5,TYPE2|5,25.0,TYPMR4-1,edu.1.4,Male,None,None,None,...,5.0,Villiers-le-Bel,Chef-lieu canton,95,None,fede.106,fede.106,1.00,Fédération Française de boxe,Fédération unisport olympique
49990,True,csp_8_4,TYPE2|2,19.0,TYPMR4-2,edu.1.4,Female,None,None,None,...,5.0,Villiers-le-Bel,Chef-lieu canton,95,None,fede.115,fede.115,1.00,Fédération Française de handball,Fédération unisport olympique
49991,False,csp_4_7,TYPE1|1,48.0,TYPMR4-1,edu.1.7,Male,ct_9,O,tr_1,...,4.0,Villiers-le-Bel,Chef-lieu canton,95,COND.1.6,fede.105,fede.105,1.00,Fédération Française de basketball,Fédération unisport olympique


In [15]:
# J'ai l'impression que certaines catégories n'étaient pas bien comprises. Il aurait fallu que j'écrive "None", pour la colonne Catégorie.

In [ ]:
# Faire le nécessaire pour round up si cela ne marche pas.

In [ ]:
#Voir ici les colonnes que je garde, ou non.

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn
sklearn.set_config(display="diagram")
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)